In [30]:
import geopandas as gpd;
import geopy.distance;
import pandas as pd;
import numpy as np;

In [ ]:
# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)

# print(geopy.distance.geodesic(coords_1, coords_2).km)

In [13]:
#gis_osm_poi = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pois_free_1.shp')
gis_osm_poi = gpd.read_file('../GeoJsonFiles/school_point.geojson');
poi_crs = gis_osm_poi.to_crs(epsg=3857);

gis_osm_poi['longitude'] = gis_osm_poi['geometry'].x;
gis_osm_poi['latitude'] = gis_osm_poi['geometry'].y;

In [7]:
#poi_crs

In [14]:
gis_osm_poi.head()

,amenity,name,ele,addr:housenumber,addr:street,operator,website,old_name,geometry,longitude,latitude
0,school,Ames High School,None,1921,Ames High Dive,None,None,None,POINT (-93.63426 42.04034),-93.634264,42.040345
1,school,Ames Middle School,298,None,None,None,None,None,POINT (-93.67106 42.01321),-93.671057,42.013210
2,school,Beardshear School,283,None,None,None,None,None,POINT (-93.60848 42.03070),-93.608478,42.030704
3,school,Camp Sawyer Elementary School,308,3622,Woodland Street,Ames Community School District,http://www.ames.k12.ia.us/schools/sawyer-eleme...,Edwards School,POINT (-93.66855 42.02582),-93.668553,42.025819
4,school,Edwards Elementary School,None,820,Miller Avenue,Ames Community School District,http://www.ames.k12.ia.us/schools/edwards-elem...,None,POINT (-93.68563 42.01595),-93.685633,42.015945


In [36]:

coords_1 = (gis_osm_poi["latitude"][0], gis_osm_poi["longitude"][0]);
coords_2 = (gis_osm_poi["latitude"][1], gis_osm_poi["longitude"][1]);

print(geopy.distance.geodesic(coords_1, coords_2).miles > 4);

# from geopy.distance import geodesic

False


In [8]:
poi_crs["geometry"]

0    POINT (-10423318.589 5167024.832)
1    POINT (-10427414.334 5162958.413)
2    POINT (-10420448.060 5165579.895)
3    POINT (-10427135.634 5164847.775)
4    POINT (-10429036.938 5163368.227)
5    POINT (-10422651.763 5167429.004)
6    POINT (-10421569.437 5168511.649)
7    POINT (-10422435.314 5164265.093)
Name: geometry, dtype: geometry

In [29]:
housing_coords = pd.read_csv('../data/house_coordinates.csv', index_col = 0);
housing_coords.head()

,PID,Address,Coords4,latitude,longitude
0,909176150,"436 HAYWARD AVE, Ames, Iowa, USA","436 Hayward Ave, Ames, IA 50014, USA",42.017780,-93.651452
1,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186
2,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186
3,911128020,"320 S 2ND ST, Ames, Iowa, USA","320 S 2nd St, Ames, IA 50010, USA",42.021389,-93.614855
4,535377150,"1524 DOUGLAS AVE, Ames, Iowa, USA","1524 Douglas Ave, Ames, IA 50010, USA",42.038070,-93.612065


In [94]:
def group_vincinity(df_house, df_place, colname="POI", mileage = 0.25):
    '''
    Input variables:
    df_house: Dataframe of the houses. Need to contain the columns ["longitude", "latitude"].
    df_place: Dataframe of places of interests. Need to contain the columns ["longitude", "latitude"].
    colname: A string, indicates the column name which will be attached to df_house.
    mileage: A float. Will look for all pairs of df_house and df_place within th mileage miles.
    
    Output variables:
    An updated dataframe of df_house which has a new column "POI".
    The column consists integers which indicates the number of 'df_place' within 'mileage' in each 'df_house'.
    '''
    df_house["gps"] = df_house[["latitude", "longitude"]].apply(lambda x: (x["latitude"], x["longitude"]), axis = 1);
    df_house[colname] = df_house[["gps"]].apply(lambda x: np.sum(df_place[["latitude", "longitude"]].\
    apply(lambda y: geopy.distance.geodesic((y["latitude"],y["longitude"]), x).miles < mileage, axis = 1)), axis = 1);
    df_house.drop("gps", axis = 1, inplace = True);
    return df_house;

In [96]:
housing_coords.head(30)

,PID,Address,Coords4,latitude,longitude,POI
0,909176150,"436 HAYWARD AVE, Ames, Iowa, USA","436 Hayward Ave, Ames, IA 50014, USA",42.017780,-93.651452,7
1,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186,6
2,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186,6
3,911128020,"320 S 2ND ST, Ames, Iowa, USA","320 S 2nd St, Ames, IA 50010, USA",42.021389,-93.614855,5
4,535377150,"1524 DOUGLAS AVE, Ames, Iowa, USA","1524 Douglas Ave, Ames, IA 50010, USA",42.038070,-93.612065,5
5,534177230,"2304 FILLMORE AVE, Ames, Iowa, USA","2304 Fillmore Ave, Ames, IA 50010, USA",42.044900,-93.631893,6
6,908128060,"4003 MARIGOLD DR, Ames, Iowa, USA","4003 Marigold Dr, Ames, IA 50014, USA",42.019109,-93.671613,5
7,902135020,"212 10TH ST 210, Ames, Iowa, USA","212 10th St, Ames, IA 50010, USA",42.031052,-93.612896,5
8,528228540,"2431 LAKESIDE DR, Ames, Iowa, USA","2431 Lakeside Dr, Ames, IA 50010, USA",42.063401,-93.642741,3
9,923426010,"602 KEN MARIL RD, Ames, Iowa, USA","602 Ken Maril Rd, Ames, IA 50010, USA",41.986336,-93.603617,0


In [95]:
%time group_vincinity(housing_coords, gis_osm_poi, "POI", 2.5);
# housing_coords.drop("gps", axis = 1, inplace = True)

Wall time: 16.8 s


,PID,Address,Coords4,latitude,longitude,POI
0,909176150,"436 HAYWARD AVE, Ames, Iowa, USA","436 Hayward Ave, Ames, IA 50014, USA",42.017780,-93.651452,7
1,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186,6
2,905476230,"3416 WEST ST, Ames, Iowa, USA","3416 West St, Ames, IA 50014, USA",42.024697,-93.664186,6
3,911128020,"320 S 2ND ST, Ames, Iowa, USA","320 S 2nd St, Ames, IA 50010, USA",42.021389,-93.614855,5
4,535377150,"1524 DOUGLAS AVE, Ames, Iowa, USA","1524 Douglas Ave, Ames, IA 50010, USA",42.038070,-93.612065,5
...,...,...,...,...,...,...
2619,903205040,"1021 RIDGEWOOD AVE, Ames, Iowa, USA","1021 Ridgewood Ave, Ames, IA 50010, USA",42.031937,-93.626510,6
2620,905402060,"3619 MARY CIR, Ames, Iowa, USA","3619 Mary Cir, Ames, IA 50014, USA",42.027798,-93.666899,6
2621,909275030,"2140 SUNSET DR 2142, Ames, Iowa, USA","2140 Sunset Dr, Ames, IA 50014, USA",42.019944,-93.643206,8
2622,907192040,"5319 CLEMENS BLVD, Ames, Iowa, USA","5319 Clemens Blvd, Ames, IA 50014, USA",42.016826,-93.690382,3
